In [ ]:
import serial
import time
import threading
import cv2
import os
from pathlib import Path
from datetime import datetime

# ---------------- MOTOR ----------------
puerto = "/dev/ttyUSB0"  # ESP32
baudrate = 115200

def ejecutar_motor():
    try:
        esp = serial.Serial(puerto, baudrate, timeout=1)
        time.sleep(2)  # esperar reinicio
        print("Conectado al ESP32 en", puerto)
    except Exception as e:
        print("Error al conectar:", e)
        return

    # Enviar comando para activar motor
    esp.write(b"motor\n")
    print("Comando 'motor' enviado al ESP32.")

    # Leer la respuesta del ESP32
    while True:
        if esp.in_waiting:
            try:
                respuesta = esp.readline().decode('utf-8', errors='ignore').strip()
                if respuesta:
                    print(respuesta)
                    if "Motor terminado" in respuesta:
                        break
            except Exception as e:
                print("Error leyendo Serial:", e)
        else:
            time.sleep(0.1)

    esp.close()
    print("Puerto serial cerrado.")

# ---------------- CÁMARA ----------------
CAPTURE_COUNT = 20  # número de imágenes a capturar
INTERVAL = 1        # segundos entre capturas
MODEL_FOLDER = Path('.')            
MODEL_FILE_NAME = 'best.pt'      
MODEL_PATH = (MODEL_FOLDER / MODEL_FILE_NAME).resolve()
if not MODEL_PATH.exists():
    raise FileNotFoundError(f'No se encontró el archivo del modelo en {MODEL_PATH}. Ajusta MODEL_FILE_NAME o mueve el archivo junto al notebook.')
print('Usando exclusivamente el modelo:', MODEL_PATH.name)

def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def capturar_imagenes():
    # Crear carpeta timestamp
    global carpeta 
    carpeta = datetime.now().strftime('%Y%m%d_%H%M%S')
    ensure_dir(carpeta)

    # Abrir cámara
    cap = cv2.VideoCapture(2)
    if not cap.isOpened():
        raise RuntimeError('No se pudo abrir la cámara. Verifica que tienes una cámara conectada y permisos.')

    for i in range(CAPTURE_COUNT):
        ret, frame = cap.read()
        if ret:
            nombre_archivo = os.path.join(carpeta, f'imagen_{i+1}.jpg')
            cv2.imwrite(nombre_archivo, frame)
            print(f'Imagen {i+1}/{CAPTURE_COUNT} guardada -> {nombre_archivo}')
        else:
            print(f'Frame {i+1} no leído')
        time.sleep(INTERVAL)

    cap.release()
    cv2.destroyAllWindows()
    print(f'Proceso completado. Imágenes guardadas en: {carpeta}')

# ---------------- EJECUCIÓN CONCURRENTE ----------------
# Crear hilos
hilo_motor = threading.Thread(target=ejecutar_motor)
hilo_camara = threading.Thread(target=capturar_imagenes)

# Iniciar hilos
hilo_motor.start()
hilo_camara.start()

# Esperar que ambos terminen
hilo_motor.join()
hilo_camara.join()

print("Motor y captura de imágenes completados.")
